In [48]:
import pandas as pd

# 2) Of the 3 billion Swiss francs in storm damage recorded in 2005, 2,990 million Swiss francs were due to the large-scale flooding of 21/22 August.

# Total damages per year
df = pd.read_csv('../data/total_damage.csv', header=None)
df.columns
df = df.drop(df.columns[[2, 4, 3, 5]], axis=1)
df.columns = df.iloc[1]
df = df.iloc[2:].reset_index(drop=True)
df = df.rename(columns={'Year': 'year'})
df = df.set_index("year")
df.head(10)

1,Amount (billions)
year,
1972,47
1973,241
1974,68
1975,340
1976,40
1977,539
1978,"1,103"
1979,94
1980,49


In [49]:
dd = pd.read_csv('../data/disaster_deaths.csv', sep=';', na_values=['', 'NaN', 'nan'])
dd = dd.fillna(0)
dd = dd.iloc[:, 0:].astype(int)

In [44]:
# total deaths per year
sums_year = dd[['year', 'total']]
sums_year = sums_year.set_index("year")
sums_year

,total
year,
1946,9
1947,11
1948,8
1949,6
1950,12
...,...
2019,6
2020,2
2021,3


In [99]:
import numpy as np
df.index = df.index.astype(int)

merged_df = (sums_year.merge(df, left_index=True, right_index=True, how='outer')
             .reset_index()
             .rename(columns={'index': 'year', 'total': 'deaths', 'Amount (billions)': 'damages'})
             .assign(deaths=lambda x: x['deaths'].fillna(-1)  # Replace NaN with -1 temporarily
                                       .round()               # Round to nearest integer
                                       .astype('Int64')       # Convert to Int64 to handle missing values
                                       .replace(-1, pd.NA)    # Replace -1 back to NA
                                       .astype(str)           # Convert to string
                                       .replace('<NA>', '') 
             )
)
merged_df['damages'] = (merged_df['damages']
                        .replace('NaN', np.nan)  # Replace 'NaN' string with np.nan
                        .astype(str)             # Convert to string
                        .replace('nan', ''))
merged_df['datetime'] = pd.to_datetime(merged_df['year'].astype(str) + '-01-01')
merged_df
#merged_df.to_csv("../data/clean/merged_disasters.csv", index=False)

,year,deaths,damages,datetime
0,1946,9,,1946-01-01
1,1947,11,,1947-01-01
2,1948,8,,1948-01-01
3,1949,6,,1949-01-01
4,1950,12,,1950-01-01
...,...,...,...,...
73,2019,6,87,2019-01-01
74,2020,2,42,2020-01-01
75,2021,3,471,2021-01-01
76,2022,0,46,2022-01-01


In [91]:
# total deaths per disaster
sums = dd.drop(columns = ['year', 'total'])
sums = sums.sum()
sums = sums.reset_index()
sums.columns = ['disaster', 'deaths']
#sums.to_csv("../data/clean/disaster_deaths.csv", index=False)

,disaster,deaths
0,flood,125
1,debris flow,24
2,landslide,54
3,rockfall,96
4,windstrom,112
5,lightning,165
6,ice avalanche,88
7,earthquake,3
8,other processes,4
